In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
!pip install BeautifulSoup4
!pip install requests

In [3]:
from bs4 import BeautifulSoup

Webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(Webpage.text, 'lxml')

data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        data.append(section)

toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


###  Ignore cells with a borough that is Not assigned.

In [4]:
toronto_df.shape

(288, 3)

In [5]:
toronto_df.Borough.head()

0        Not assigned
1        Not assigned
2          North York
3          North York
4    Downtown Toronto
Name: Borough, dtype: object

In [6]:
Torontoclean_df = toronto_df[toronto_df.Borough != 'Not assigned']
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Index are reset

In [7]:
Torontoclean_df = Torontoclean_df.reset_index(drop=True)
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
Torontoclean_df.shape

(211, 3)

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
Torontoclean_df[Torontoclean_df.Neighbourhood == 'Not assigned'].count()

Postcode         1
Borough          1
Neighbourhood    1
dtype: int64

In [10]:
Torontoclean_df['Neighbourhood'].replace("Not assigned", Torontoclean_df["Borough"],inplace=True)
Torontoclean_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
Torontoclean_df[Torontoclean_df.Neighbourhood == 'Not assigned'].count()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

In [12]:
Torontoclean_df.groupby("Postcode")["Neighbourhood"].sum().count()

103

### We here try to manage to use comma between different neighbourhood

In [13]:
cluster_toronto = Torontoclean_df.groupby("Postcode").Neighbourhood.agg([('Neighbourhood', ', '.join)])

In [14]:
cluster_toronto.head()

,Neighbourhood
Postcode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae


In [15]:
cluster_toronto['Borough'] = Torontoclean_df.groupby("Postcode").Borough.agg([('Borough', ', '.join)])
cluster_toronto.head()

,Neighbourhood,Borough
Postcode,,
M1B,"Rouge, Malvern","Scarborough, Scarborough"
M1C,"Highland Creek, Rouge Hill, Port Union","Scarborough, Scarborough, Scarborough"
M1E,"Guildwood, Morningside, West Hill","Scarborough, Scarborough, Scarborough"
M1G,Woburn,Scarborough
M1H,Cedarbrae,Scarborough


In [16]:
new = cluster_toronto["Borough"].str.split(',', n = 1, expand = True)

In [17]:
cluster_toronto["Borough"] = new[0]

In [18]:
columnsTitles=["Borough","Neighbourhood"]
cluster_toronto=cluster_toronto.reindex(columns=columnsTitles)

In [19]:
cluster_toronto.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [20]:
cluster_toronto.shape

(103, 2)